This notebook measures the intensities and errors of CO 2-1 emission at the locations of all of the SNe in our sample across all of the resolutions in the survey. This is used to prepare to answer the question in section 3.1 Do we see gas at supernova locations? 

We will use this to uncover what resolution will be best for this study.

In [1]:
# IMPORT PACKAGES

#global
import numpy as np
from matplotlib import pyplot as plt
import astropy
import astropy.io.fits as pyfits
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.io import ascii
from astropy.io import fits
import os

#local
import sys
sys.path.append('/home/mayker.1/Desktop/NessPythonFunctions')

from findResolution import findRes
from getMapValue    import getValue
from nonZeroError   import findErrVals
from nonZeroError   import nonZeroErrArray
from nonZeroError   import findSignal

In [2]:
# DEFINE FUNCTIONS

def genFileName(galaxy, mapType, res, telOrient):
    
    """
    Generates the filenames for the CO maps.
    
    Parameters
    ----------
    galaxy    : string : name of galaxy
    mapType   : string : "int", "err", or "EW"   
    res       : string : resolution of map ("" (native), "_60pc", "_90pc", "_120pc", "_150pc", etc.)
    telOrient : string : 12m+7m+tp, 7m+tp, etc.
    
    Returns
    -------
    fileName : string : full path to file on Tycho.
        (/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc2997/ngc2997_12m+7m+tp_co21_broad_mom0_150pc.fits)

    """
    
    if mapType == "int":
        mapTypeStr = "_broad_mom0"
    elif mapType == "err":
        mapTypeStr = "_broad_emom0"
    elif mapType  == "EW":
        mapTypeStr = "_strict_ew"
    else:
        print("Wrong Map Type.")
        
    preamble = "/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" 
    
    fileName = preamble + galaxy + "/" + galaxy + "_" + telOrient + "_co21" + res + mapTypeStr + ".fits"
    return(fileName)


def getAlphaCO(galaxy, SNra, SNdec):
    """
    Pulls the metallicity dependent Alpha CO value at the SN location using Sun+2020 maps
    or assigns a value of alpha CO = 4.35/0.65 if no Sun value is available.
    """
        
    GalName = galaxy.strip().upper()
    
    if GalName == 'CIRCINUS':
        GalName = 'ESO097-013'
    
    fileName = "/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/" + GalName + '_alphaCO21_Sun+20.fits'

    if(os.path.isfile(fileName) == True):
        alphaCO = getValue(fileName,float(SNra),float(SNdec))
    else:
        alphaCO = 4.35/0.65
    
    return(alphaCO)

def findSNRErr(errorArray, i, res):
    """
    Finds the lowest non-zero error for use in calculating the SNR in the map if the pixel has a non-real error associated with it.
    Otherwise just uses the pixel error.
    """
    
    if(errorArray[i] <= 0.0):   
        hdulist = pyfits.open(genFileName(galaxies[i], "err", res, telOrient[i]))
        map = hdulist[0].data 
        flatMap = map.flatten()
        flatMap = flatMap[flatMap > 0.0]
        flatMap.sort()
        usedErr = flatMap[0]
    else: 
        usedErr = errorArray[i]
    
    return(usedErr)
    

In [3]:
# import SNeCO data

dataFile = '../Data/3.SNe+GalData.csv'
dataTable = Table.read(dataFile, format='csv') 
#dataTable.colnames
#dataTable


In [4]:
# pull galaxy names and telescope orientation to generate the file lists

galaxies  = dataTable['galaxy']
telOrient = dataTable['telOrient']
SNname    = dataTable['SN_name']
SNra      = dataTable['SN_ra']
SNdec     = dataTable['SN_dec']
beamsize  = dataTable['map_beamsize']
telOrient = dataTable['telOrient']
distance  = dataTable['dist']


In [5]:
#Create Intensity Measurement Database 

intenData = Table({'galaxy'       : galaxies,
              'SN_name'     : SNname,
              'NatRes'      : np.zeros(len(galaxies)),
              'NatInt'      : np.zeros(len(galaxies)),
              'pc60Int'     : np.zeros(len(galaxies)),
              'pc90Int'     : np.zeros(len(galaxies)),
              'pc120Int'    : np.zeros(len(galaxies)),
              'pc150Int'    : np.zeros(len(galaxies)),
              'pc500Int'    : np.zeros(len(galaxies)),
              'pc750Int'    : np.zeros(len(galaxies)),
              'pc1000Int'   : np.zeros(len(galaxies)),
              'pc1250Int'   : np.zeros(len(galaxies)),
              'NatErr'      : np.zeros(len(galaxies)),
              'pc60Err'     : np.zeros(len(galaxies)),
              'pc90Err'     : np.zeros(len(galaxies)),
              'pc120Err'    : np.zeros(len(galaxies)),
              'pc150Err'    : np.zeros(len(galaxies)),
              'pc500Err'    : np.zeros(len(galaxies)),
              'pc750Err'    : np.zeros(len(galaxies)),
              'pc1000Err'   : np.zeros(len(galaxies)),
              'pc1250Err'   : np.zeros(len(galaxies)),
              'NatEW'       : np.zeros(len(galaxies)),
              'pc60EW'      : np.zeros(len(galaxies)),
              'pc90EW'      : np.zeros(len(galaxies)),
              'pc120EW'     : np.zeros(len(galaxies)),
              'pc150EW'     : np.zeros(len(galaxies)),
              'pc500EW'     : np.zeros(len(galaxies)),
              'pc750EW'     : np.zeros(len(galaxies)),
              'pc1000EW'    : np.zeros(len(galaxies)),
              'pc1250EW'    : np.zeros(len(galaxies)),
              'NatSNR'      : np.zeros(len(galaxies)),
              'pc60SNR'     : np.zeros(len(galaxies)),
              'pc90SNR'     : np.zeros(len(galaxies)),
              'pc120SNR'    : np.zeros(len(galaxies)),
              'pc150SNR'    : np.zeros(len(galaxies)),
              'pc500SNR'    : np.zeros(len(galaxies)),
              'pc750SNR'    : np.zeros(len(galaxies)),
              'pc1000SNR'   : np.zeros(len(galaxies)),
              'pc1250SNR'   : np.zeros(len(galaxies)),
              'alphaCO'     : np.zeros(len(galaxies)),
                })


In [6]:
# populate database

for i in range(len(galaxies)):
    
    intenData['NatRes'][i]    = findRes(float(beamsize[i]),float(distance[i]))
    
    intenData['NatInt'][i]    = getValue(genFileName(galaxies[i], "int", "", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc60Int'][i]   = getValue(genFileName(galaxies[i], "int", "_60pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc90Int'][i]   = getValue(genFileName(galaxies[i], "int", "_90pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc120Int'][i]  = getValue(genFileName(galaxies[i], "int", "_120pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc150Int'][i]  = getValue(genFileName(galaxies[i], "int", "_150pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc500Int'][i]  = getValue(genFileName(galaxies[i], "int", "_500pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc750Int'][i]  = getValue(genFileName(galaxies[i], "int", "_750pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc1000Int'][i] = getValue(genFileName(galaxies[i], "int", "_1000pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc1250Int'][i] = getValue(genFileName(galaxies[i], "int", "_1250pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    
    intenData['NatErr'][i]    = getValue(genFileName(galaxies[i], "err", "", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc60Err'][i]   = getValue(genFileName(galaxies[i], "err", "_60pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc90Err'][i]   = getValue(genFileName(galaxies[i], "err", "_90pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc120Err'][i]  = getValue(genFileName(galaxies[i], "err", "_120pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc150Err'][i]  = getValue(genFileName(galaxies[i], "err", "_150pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc500Err'][i]  = getValue(genFileName(galaxies[i], "err", "_500pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc750Err'][i]  = getValue(genFileName(galaxies[i], "err", "_750pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc1000Err'][i] = getValue(genFileName(galaxies[i], "err", "_1000pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc1250Err'][i] = getValue(genFileName(galaxies[i], "err", "_1250pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    
SET USED ERR HERE CALL FUNCTION
    
    
    intenData['NatEW'][i]     = getValue(genFileName(galaxies[i], "EW", "", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc60EW'][i]    = getValue(genFileName(galaxies[i], "EW", "_60pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc90EW'][i]    = getValue(genFileName(galaxies[i], "EW", "_90pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc120EW'][i]   = getValue(genFileName(galaxies[i], "EW", "_120pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc150EW'][i]   = getValue(genFileName(galaxies[i], "EW", "_150pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc500EW'][i]   = getValue(genFileName(galaxies[i], "EW", "_500pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc750EW'][i]   = getValue(genFileName(galaxies[i], "EW", "_750pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc1000EW'][i]  = getValue(genFileName(galaxies[i], "EW", "_1000pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))
    intenData['pc1250EW'][i]  = getValue(genFileName(galaxies[i], "EW", "_1250pc", telOrient[i]),float(SNra[i]),float(SNdec[i]))

    intenData['NatSNR'][i]    = findSignal(intenData['NatInt'][i],    findSNRErr(intenData['NatErr'], i, ""))
    intenData['pc60SNR'][i]   = findSignal(intenData['pc60Int'][i],   findSNRErr(intenData['pc60Err'], i, "_60pc"))
    intenData['pc90SNR'][i]   = findSignal(intenData['pc90Int'][i],   findSNRErr(intenData['pc90Err'], i, "_90pc"))
    intenData['pc120SNR'][i]  = findSignal(intenData['pc120Int'][i],  findSNRErr(intenData['pc120Err'], i, "_120pc"))
    intenData['pc150SNR'][i]  = findSignal(intenData['pc150Int'][i],  findSNRErr(intenData['pc150Err'], i, "_150pc"))
    intenData['pc500SNR'][i]  = findSignal(intenData['pc500Int'][i],  findSNRErr(intenData['pc500Err'], i, "_500pc"))
    intenData['pc750SNR'][i]  = findSignal(intenData['pc750Int'][i],  findSNRErr(intenData['pc750Err'], i, "_750pc"))
    intenData['pc1000SNR'][i] = findSignal(intenData['pc1000Int'][i], findSNRErr(intenData['pc1000Err'], i, "_1000pc"))
    intenData['pc1250SNR'][i] = findSignal(intenData['pc1250Int'][i], findSNRErr(intenData['pc1250Err'], i, "_1250pc"))
        
    intenData['alphaCO'][i]   = getAlphaCO(galaxies[i], float(SNra[i]), float(SNdec[i]))    


In [7]:
intenData


galaxy,SN_name,NatRes,NatInt,pc60Int,pc90Int,pc120Int,pc150Int,pc500Int,pc750Int,pc1000Int,pc1250Int,NatErr,pc60Err,pc90Err,pc120Err,pc150Err,pc500Err,pc750Err,pc1000Err,pc1250Err,NatEW,pc60EW,pc90EW,pc120EW,pc150EW,pc500EW,pc750EW,pc1000EW,pc1250EW,NatSNR,pc60SNR,pc90SNR,pc120SNR,pc150SNR,pc500SNR,pc750SNR,pc1000SNR,pc1250SNR,alphaCO
str8,str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
circinus,SN1996cr,148.96539476404914,186.84803771972656,nan,nan,nan,186.84803771972656,111.39796447753906,97.68943786621094,nan,nan,2.1378190517425537,nan,nan,nan,2.1378190517425537,0.24664470553398132,0.17664189636707306,nan,nan,34.56536865234375,nan,nan,nan,34.56536865234375,49.76144790649414,69.42247772216797,nan,nan,0.0,nan,nan,nan,0.0,0.0,0.0,nan,nan,5.011212121095752
ngc0253,SN1940E,150.1701284517627,219.3036651611328,nan,nan,nan,219.3036651611328,153.83908081054688,153.87240600585938,nan,nan,1.465811014175415,nan,nan,nan,1.465811014175415,0.18237826228141785,0.11332354694604874,nan,nan,45.862266540527344,nan,nan,nan,45.862266540527344,60.678550720214844,65.11054229736328,nan,nan,0.0,nan,nan,nan,0.0,0.0,0.0,nan,nan,4.478273076921023
ngc0300,SN2019qyl,82.84239405142596,-0.6273980736732483,nan,-0.6273980736732483,-0.3393101096153259,-0.10330438613891602,0.502941370010376,0.46108144521713257,nan,nan,0.49926596879959106,nan,0.49926596879959106,0.24836020171642303,0.1649596244096756,0.03243602439761162,0.025462361052632332,nan,nan,nan,nan,nan,nan,nan,6.742672920227051,6.879421234130859,nan,nan,0.0,nan,0.0,0.0,0.0,0.0,0.0,nan,nan,9.413450498015168
ngc0628,SN2019krl,53.490181864724654,0.6559359431266785,0.7540172934532166,0.963236927986145,0.9129124879837036,0.762334406375885,0.4191957712173462,0.4084402322769165,nan,nan,1.2892513275146484,1.097657322883606,0.7536418437957764,0.5805757641792297,0.46779823303222656,0.13918526470661163,0.08817216753959656,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,nan,6.892366043319281
ngc0628,SN2013ej,53.490181864724654,5.771303653717041,4.944133281707764,2.7272579669952393,1.8072082996368408,1.341525912284851,0.6933073997497559,0.5306633114814758,nan,nan,1.9949342012405396,1.7279061079025269,1.2605571746826172,1.0164546966552734,0.8571658730506897,0.18906767666339874,0.1100897341966629,nan,nan,nan,nan,nan,nan,nan,nan,1.9705547094345093,nan,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,nan,7.274922328337008
ngc1068,SN2018ivc,588.6196760664847,127.7774887084961,nan,nan,nan,nan,nan,128.53480529785156,nan,nan,0.6721417903900146,nan,nan,nan,nan,nan,0.4782250225543976,nan,nan,32.07529830932617,nan,nan,nan,nan,nan,36.46405792236328,nan,nan,0.0,nan,nan,nan,nan,nan,0.0,nan,nan,6.692307692307692
ngc1087,SN1995V,123.1488071983115,15.466903686523438,nan,nan,15.466903686523438,12.753849029541016,4.536659240722656,4.371304512023926,nan,nan,1.1405595541000366,nan,nan,1.1405595541000366,0.9069606065750122,0.2801399230957031,0.1639837622642517,nan,nan,6.007981300354004,nan,nan,6.007981300354004,6.287197589874268,8.803016662597656,14.90282154083252,nan,nan,0.0,nan,nan,0.0,0.0,0.0,0.0,nan,nan,6.6412306805812955
ngc1097,SN1999eu,111.66580536484963,0.0,nan,nan,0.0,0.0,0.0,0.0,nan,nan,0.7541713714599609,nan,nan,0.7541713714599609,0.5342973470687866,0.12150260806083679,0.06001226231455803,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.0,nan,nan,0.0,0.0,0.0,0.0,nan,nan,10.910709514032963
ngc1097,SN1992bd,111.66580536484963,282.0977783203125,nan,nan,282.0977783203125,260.3741149902344,223.9143524169922,193.74581909179688,nan,nan,2.054412841796875,nan,nan,2.054412841796875,1.491909384727478,0.4566611051559448,0.2543865442276001,nan,nan,46.925418853759766,nan,nan,46.925418853759766,49.31303024291992,98.3154525756836,110.4483871459961,nan,nan,0.0,nan,nan,0.0,0.

In [8]:
ascii.write(intenData, '../Data/3.IntenData.csv', format='csv', fast_writer=False, overwrite=True)  